In [1]:
import pandas as pd
import geopandas as gpd
import streamlit as st 
import plotly.express as px
import folium
from streamlit_folium import st_folium, folium_static
from folium.plugins import MarkerCluster

In [2]:
data_1 = r"D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\ETL\siniestros_por_comuna.parquet"  ### Siniestros por comuna
data_2 =  r"D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\ETL\siniestros.parquet"  ### Siniestros por tipo
data_3 = r"D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\comunas.geojson"  ### Geodata


In [3]:
df1 = pd.read_parquet(data_1)

In [3]:
df2 = pd.read_parquet(data_2)

In [4]:
df3 = gpd.read_file(data_3)

In [5]:
df3.head(3)

,ID,OBJETO,COMUNAS,BARRIOS,PERIMETRO,AREA,geometry
0,1,LIMITE COMUNAL,2.0,RECOLETA,21452.838648,6.317265e+06,"MULTIPOLYGON (((-58.38000 -34.57002, -58.38002..."
1,3,LIMITE COMUNAL,6.0,CABALLITO,10990.964471,6.851029e+06,"MULTIPOLYGON (((-58.43061 -34.60705, -58.43056..."
2,6,LIMITE COMUNAL,10.0,FLORESTA - MONTE CASTRO - VELEZ SARSFIELD - VE...,18332.037457,1.265656e+07,"MULTIPOLYGON (((-58.48834 -34.62016, -58.48820..."


In [6]:
min_lon, max_lon = -58.23, -58.63
min_lat, max_lat = -34.75, -34.50

In [7]:
m = folium.Map(location=[-34.61777, -58.43210], 
            tiles='Esri_WorldGrayCanvas', 
            zoom_control=False, zoom_start=12, min_zoom=12,
            max_bounds=True, # Creation of map's limits.
            min_lat = min_lat,
            max_lat=max_lat,
            min_lon=min_lon,
            max_lon=max_lon                  
            )

marker_cluster = MarkerCluster().add_to(m)

In [8]:
st_data = st_folium(m, width=700, height=650)

2024-04-21 01:32:16.787 
  command:

    streamlit run d:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [6]:
df1.head(3)

,COMUNA,Poblacion 2021,Poblacion 2020,Total victimas 2020,Total victimas 2021
0,1,257235,256405,10.0,9.0
1,2,149371,149430,5.0,4.0
2,3,193425,193276,10.0,8.0


In [9]:
### Calculo kpi total de siniestros Buenos aires
 # total Accidentes mortales 2021 = am21
 # total Accidentes mortales 2020 = am20
am21 = df1['Total victimas 2021'].sum()
am20 = df1['Total victimas 2020'].sum()
print(am21)

96.0


In [10]:
kpi_am21 = (am21-am20)/am20*100
print(kpi_am21)

21.518987341772153


### Muertes por cada 100,000 habitantes

muertes por cada 100000 habitantes = ((total muertes)/(poblacion ciudad))*100000

In [16]:
#m2021 = muertes por cada 100000 habitantes.
m2021 = ((df1['Total victimas 2021'].sum())/(df1['Poblacion 2021'].iloc[0]))*100000

In [14]:
m2020 = ((df1['Total victimas 2020'].sum())/(df1['Poblacion 2020'].iloc[0]))*100000

In [17]:
# Calculo de kpi en relacion al anio anterior
m100 = m2021 - m2020

In [18]:
m100 = round(m100, 0)

In [19]:
print(m100)

7.0


In [21]:
df2

,ID,N_VICTIMAS,AAAA,COMUNA,Longitud,Latitud,VICTIMA
index,,,,,,,
512,2020-0001,1.0,2020,15,-58.439104,-34.592134,PEATON
513,2020-0002,1.0,2020,7,-58.450194,-34.646464,MOTO
514,2020-0003,1.0,2020,3,-58.393376,-34.609276,PEATON
515,2020-0004,1.0,2020,1,-58.390556,-34.609193,PEATON
516,2020-0005,1.0,2020,3,-58.410370,-34.624770,MOTO
...,...,...,...,...,...,...,...
679,2021-0093,1.0,2021,7,-58.433538,-34.645616,MOTO
680,2021-0094,1.0,2021,9,-58.467398,-34.651178,MOTO
681,2021-0095,1.0,2021,11,-58.472934,-34.619847,MOTO


### Muertes por tipo de victima

In [4]:
df_vic= df2[['AAAA','COMUNA','VICTIMA','N_VICTIMAS']]

In [5]:
df_vic = df_vic[df_vic['AAAA'] == 2021]

In [6]:
df_vic = df_vic.drop('AAAA', axis=1)

In [7]:
df_vic.head(3)

,COMUNA,VICTIMA,N_VICTIMAS
index,,,
588,1,MOTO,1.0
589,9,MOTO,1.0
590,7,MOTO,1.0


In [8]:
df_vic = df_vic.groupby(['COMUNA','VICTIMA'])['N_VICTIMAS'].sum().reset_index()

In [9]:
df_vic.head(5)

,COMUNA,VICTIMA,N_VICTIMAS
0,1,BICICLETA,1.0
1,1,MOTO,4.0
2,1,PEATON,4.0
3,2,AUTO,1.0
4,2,BICICLETA,1.0


In [10]:
df_comunas = df_vic[df_vic['COMUNA'] == 1]

In [11]:
df_grouped = df_comunas.groupby('VICTIMA')['N_VICTIMAS'].sum().reset_index()

In [22]:
#data_comuna = px.data.df_vic().query("COMUNA == 1")
data = df_grouped
fig = px.bar(data, x='VICTIMA', y='N_VICTIMAS', color='VICTIMA')

fig.update_layout(showlegend=False)
fig.show()